# ImportiImporting and preparing rental apartments dataeren

## Libraries and settings

In [16]:
# Libraries
import os
import re
import time
import fnmatch
import numpy as np
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Get current working directory
print(os.getcwd())

/workspaces/groceries/preparation


## CSV Letzte Zeile

In [17]:
import os
import pandas as pd

def create_prepared_csv(input_path, output_path, supermarket):
    # Überprüfen, ob die Ausgabedatei bereits existiert
    if not os.path.exists(output_path):
        # DataFrame aus der Eingabedatei erstellen
        df = pd.read_csv(input_path, sep=';')
        # Neue Spalte "Supermarkt" hinzufügen
        df['Supermarkt'] = supermarket
        # DataFrame in eine neue CSV-Datei schreiben
        df.to_csv(output_path, index=False)
        print(f"Die Datei {output_path} wurde erfolgreich erstellt.")
    else:
        print(f"Die Datei {output_path} existiert bereits.")

# Aldi Flour
create_prepared_csv('../CSV/Aldi_Flour.csv', '../CSV/prepared/prepared_CSV_Aldi_Flour.csv', 'Aldi')

# Coop Flour
create_prepared_csv('../CSV/Coop_Flour.csv', '../CSV/prepared/prepared_CSV_Coop_Flour.csv', 'Coop')

# Migros Flour
create_prepared_csv('../CSV/Migros_Flour.csv', '../CSV/prepared/prepared_CSV_Migros_Flour.csv', 'Migros')


Die Datei ../CSV/prepared/prepared_CSV_Aldi_Flour.csv existiert bereits.
Die Datei ../CSV/prepared/prepared_CSV_Coop_Flour.csv existiert bereits.
Die Datei ../CSV/prepared/prepared_CSV_Migros_Flour.csv existiert bereits.


## Merge Flour

In [18]:
import pandas as pd
import os

# Liste der Dateipfade
Flours_merge = [
    '../CSV/prepared/prepared_CSV_Aldi_Flour.csv',
    '../CSV/prepared/prepared_CSV_Coop_Flour.csv',
    '../CSV/prepared/prepared_CSV_Migros_Flour.csv'
]

# Eine leere DataFrame erstellen, um die Daten zu speichern
gesamt_df = pd.DataFrame()

# Durchlaufen Sie die Liste der Dateipfade und fügen Sie die Daten dem Gesamt-DataFrame hinzu
for Flour_merge in Flours_merge:
    df = pd.read_csv(Flour_merge, sep=';')  # Änderung hier: sep=';'
    gesamt_df = pd.concat([gesamt_df, df], ignore_index=True)  # Änderung hier: Verwendung von pd.concat

# Den Gesamt-DataFrame in eine neue CSV-Datei schreiben
gesamt_df.to_csv('Flour_merge.csv', index=False)


## Importing data

In [19]:
# Show .csv - files in the directory
flist = fnmatch.filter(os.listdir('.'), '*.csv')
for i in flist:
    print(i)

# Read the data to a pandas data frame
df = pd.read_csv('Flour_merge.csv', sep=',', encoding='utf-8')

# Show first records of data frame
df.head(50)

Egg_merge.csv
Flour_merge.csv
Butter_merge.csv


,"product_type,web-scraper-order,web-scraper-start-url,productname_raw,price_raw,weight_raw,Supermarkt"
0,"flour,1698753392-1,https://www.aldi-now.ch/de/..."
1,"flour,1698753392-2,https://www.aldi-now.ch/de/..."
2,"flour,1698753392-3,https://www.aldi-now.ch/de/..."
3,"flour,1698753392-4,https://www.aldi-now.ch/de/..."
4,"flour,1698753392-5,https://www.aldi-now.ch/de/..."
5,"flour,1698753392-6,https://www.aldi-now.ch/de/..."
6,"flour,1698753392-7,https://www.aldi-now.ch/de/..."
7,"flour,1698753392-8,https://www.aldi-now.ch/de/..."
8,"flour,1698753392-9,https://www.aldi-now.ch/de/..."
9,"flour,1698753392-10,https://www.aldi-now.ch/de..."


## Count number of rows and columns in the data frame

In [20]:
# Dimension (rows, columns)
print('Dimension:', df.shape)

# Number of rows
print('Number of rows:', df.shape[0])

# Number of columns
print('Number of columns:', df.shape[1])

Dimension: (67, 1)
Number of rows: 67
Number of columns: 1


## Get data types (raw-format from web scraping)

In [21]:
# Get data types (note that in pandas, a string is referred to as 'object')
df.dtypes

product_type,web-scraper-order,web-scraper-start-url,productname_raw,price_raw,weight_raw,Supermarkt    object
dtype: object